Edward Tufte suggests to visually indicate the location of the max and min values in a scatterplot with the lines that demark the x- and y-axis of the plot. He calls this a rangeplot. This post shows how to create a such a plot in Altair 5.

First, we create a random dataset.

In [1]:
import numpy as np
import pandas as pd

df = pd.DataFrame({
    'x': np.random.randn(100),
    'y': np.random.randn(100),
    'category': np.random.randint(0, 2, 100),
})


Then, we create a scatterplot and adjust the axes as recommended by Tufte. 
We remove grids and ticks, as is common in Tufte-style plots.
As Altair's default font size is rather small, we increase it.

In [2]:
import altair as alt

alt.Chart(df, title="Rangeplot").mark_point(filled=True).encode(
    alt.X('x',
      axis=alt.Axis(
          offset=40, 
          grid=False, 
          ticks=False, 
          labelFontSize=12,
          titleFontSize=16
      ),
      scale=alt.Scale(domain=[df.x.min(), df.x.max()], nice=False)
    ),
    alt.Y('y',
      axis=alt.Axis(
          offset=40, 
          grid=False, 
          ticks=False, 
          labelFontSize=12,
          titleFontSize=16
      ),
      scale=alt.Scale(domain=[df.y.min(), df.y.max()], nice=False)
    ),
    color=alt.Color('category:N')
).properties(
  height=300,
  width=400,
).configure_view(
    stroke=None,
)

alt.Chart(...)

This was quite some styling for a single plot. 
To simplify this in the future, we can make our lives easier by

- Factoring out the min_max_scale.
- Creating a custom theme following Tufte's advice on style. In addition we'll use a font is similar to fonts he regularly uses.
- Setting a default size for plots (which can be overridden).

In [5]:
from altair import Chart, X, Y, Color, Scale, theme

@theme.register('tufte', enable=True)
def tufte():
    font = "Georgia"
    return {
        'config': {
            "title": {
                'font': font,
                'fontSize': 24
            },
            "background": None,
            'view': {
                'stroke': None,
                'height': 300,
                'width': 400,
            },
            'mark': {'filled': True,},
            'axis': {
                'labelFontSize': 14,
                'titleFontSize': 18,
                'offset': 40,
                'grid': False,
                'ticks': False,
                'labelFont': font,
                'titleFont': font
            },
            'legend': { 
                'titleFont': font,
                'labelFont': font,
                'titleFontSize': 14,
                'labelFontSize': 12
            },
            'scale': {
                'nice': False
            }
        }
    }

min_max_scale = lambda df, col: Scale(domain=[df[col].min(), df[col].max()])

Chart(df, title="Rangeplot").mark_point().encode(
    X('x', scale=min_max_scale(df, 'x')),
    Y('y', scale=min_max_scale(df, 'y')),
    color=Color('category:N'),
)

alt.Chart(...)